The code below is based on the book "Machine Learning for Hackers", by Drew Conway and John Myles White.


# Importing Packages

We start by loading the libraries we'll need : 
* `tm` for text mining
* `ggplot2`, to plot the results of my algorithm.

In [1]:
library(tm)
library(ggplot2)

Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:httr’:

    content



Attaching package: ‘ggplot2’


The following object is masked from ‘package:NLP’:

    annotate




# Importing and Manipulating Data

The spam and ham emails are split into two folders. Here, we save the paths to these folders in the objects `spam.path` and `ham.path`. Then, we use the `dir` function to create a vector with all of the names of the files within these directories. We call these vectors `spam.docs` and `ham.docs`.

We removed the first spam mail from `spam.docs` because it didn't conform to the email format and this caused errors.

In [2]:
spam.path <- '../input/ham-and-spam-dataset/spam'
ham.path <- '../input/ham-and-spam-dataset/ham'

spam.docs <- dir(spam.path)
ham.docs <- dir(ham.path)

spam.docs <- spam.docs[seq(2,length(spam.docs),1)] # remove first spam email, which is not in email format

Now, we create a function calld `get.msg`. `get.msg` opens a file and returns the e-mail message.

We open the file in rt (read text) mode and used latin1 encoding, which supports non-ASCII characters, present in many emails.

Then, we select the email text, which begins after the first line break. The text before that space is e-mail metadata, such as the email sender, receiver and the time the email was sent. Metadata can also be used for spam prediction, but we will not use it here.

Finally, we use the `sample` function to split the ham and spam datasets into training and testing data.

In [3]:
get.msg <- function(path){
    con <- file(path, open = 'rt', encoding = 'latin1') # open file connection
    text <- readLines(con) # read file content
    msg <- text[seq(which(text=="")[1]+1,length(text),1)] # get text that begins after first line break
    close(con) # clone connection
    return(paste(msg, collapse = '\n'))  # return email message
}

all.spam <- sapply(spam.docs, function(p) get.msg(file.path(spam.path, p)))
all.ham <- sapply(ham.docs, function(p) get.msg(file.path(ham.path, p)))
                  
spam.train <- sample(all.spam, floor(0.7*length(all.spam)), replace = FALSE)
spam.test <- setdiff(all.spam, spam.train)

ham.train <- sample(all.ham, floor(0.7*length(all.ham)), replace = FALSE)
ham.test <- setdiff(all.ham, ham.train)

# Bulding a Term-Document Matrix

Now that we've extracted the email message from all email files, we will create a function to generate a Term-Document Matrix, or TDM. A TDM is an $N \times M$ matrix in which the $N$ terms found in the documents are represented in the rows and the $M$ documents are represented in the columns. The $[i,j]$ cell of this matrix corresponds to how many times the term $i$ was found in document $j$.

The `TermDocumentMatrix` function from the tm library requires two parameters : a document corpus parameter, which is a collection of documents, and a control parameter, which specifies which settings to use when building the TDM.

You can build a corups in several ways. Here, because we are using a vector of spam mails as a source, we used the `VectorSource` function into the `Corpus` function. If you want to know more about sources for bulding a corpus, enter `?getSources` at the R console.

For contol, we used `stopwords = TRUE`, which removes common English [stop words](https://en.wikipedia.org/wiki/Stop_word) from the TDM. You can enter `stopwords()` at the R console to see which are these words. We also set `removePunctiation = TRUE` and `removeNumbers = TRUE`, which are self-explanatory. Finally, we set `minDocFrequency = 2`, which assures only terms appearing more than once will show up in the TDM.

In [4]:
get.tdm <- function(doc.vec){
    doc.corpus <- Corpus(VectorSource(doc.vec))
    control <- list(stopwords = TRUE, removePunctuation = TRUE, removeNumbers = TRUE,
                   minDocFreq = 2)
    doc.dtm <- TermDocumentMatrix(doc.corpus, control)
    return(doc.dtm)
}

Now that we've created the TDM, we will build a data frame that contains the probability of each term showing up in an email, given that we know its a spam mail. We will use this information to calculate the probability of an email being spam, given that a given set of terms showed up in it.

We also calculate the term density, that is, the proportion of a specific term within the whole set of terms observed. We will not use the frequency information for classification, but it will be useful to see how these numbers compare when we consider how certain words might be affecting our results.

In [5]:
spam.tdm <- get.tdm(spam.train) # TDM = Term Document Matrix. Term (x axis) vs Document (y axis)

spam.matrix <- as.matrix(spam.tdm)
spam.counts <- rowSums(spam.matrix) # vector that contains term frequency counts
spam.df <- data.frame(names(spam.counts),
                     as.numeric(spam.counts),
                     stringAsFactors = FALSE)

names(spam.df) <- c('term', 'frequency')

spam.occurrence <- sapply(1:nrow(spam.matrix), 
 function(i) {length(which(spam.matrix[i,] > 0))/ncol(spam.matrix)}) # prop. of word occurence in documents

spam.density <- spam.df$frequency/sum(spam.df$frequency) # prop. of word occurence in words

spam.df <- transform(spam.df, density = spam.density, occurrence = spam.occurrence)

We investigate which terms are more strongly associated with spam mail by ordering the top 10 terms according to the term occurrence.

In [6]:
head(spam.df[with(spam.df, order(-occurrence)),])

,term,frequency,NA.,density,occurrence
,<chr>,<dbl>,<lgl>,<dbl>,<dbl>
11,email,582,FALSE,0.006427459,0.5600000
50,please,304,FALSE,0.003357298,0.5257143
33,list,302,FALSE,0.003335211,0.4628571
140,will,568,FALSE,0.006272847,0.4228571
392,body,266,FALSE,0.002937636,0.4142857
476,free,381,FALSE,0.004207667,0.3971429


Now, we repeat the same process for ham mail and investigate which terms are more strongly associated with them.

In [7]:
ham.tdm <- get.tdm(ham.train) # TDM = Term Document Matrix. Term (x axis) vs Document (y axis)

ham.matrix <- as.matrix(ham.tdm)
ham.counts <- rowSums(ham.matrix) # vector that contains term frequency counts
ham.df <- data.frame(cbind(names(ham.counts)),
                     as.numeric(ham.counts),
                     stringAsFactors = FALSE)

names(ham.df) <- c('term', 'frequency')

ham.df$frequency <- as.numeric(ham.df$frequency) # unnecessary?

ham.occurrence <- sapply(1:nrow(ham.matrix), 
 function(i) {length(which(ham.matrix[i,] > 0))/ncol(ham.matrix)}) # prop. of word occurence in documents
ham.density <- ham.df$frequency/sum(ham.df$frequency) # prop. of word occurence in words
ham.df <- transform(ham.df, density = ham.density, occurrence = ham.occurrence)

In [8]:
head(ham.df[with(ham.df, order(-occurrence)),])

,term,frequency,NA.,density,occurrence
,<chr>,<dbl>,<lgl>,<dbl>,<dbl>
35,list,944,FALSE,0.004701991,0.3843137
245,date,618,FALSE,0.003078210,0.3238095
1084,mailing,618,FALSE,0.003078210,0.3092437
162,can,1037,FALSE,0.005165217,0.3086835
58,wrote,646,FALSE,0.003217676,0.3030812
33,just,872,FALSE,0.004343365,0.2969188


# Building the Classifier

Finally, we build a classifier. We use the occurrence data to calculate the conditional probability of an email being spam or ham given the observed words in the email. We do this using Bayes' Theorem: 

\begin{equation}
P(spam|words) = \frac{P(words|spam)P(spam)}{P(words)}
\end{equation}

\begin{equation}
P(ham|words) = \frac{P(words|ham)P(ham)}{P(words)}
\end{equation}


Since we are only interested in the conditional probability of an email being spam in comparison to it being ham, not in its absolute value, we discard the $P(data)$ in the denominator, which is called the marginal likelihood, and use the proportional probability instead:

\begin{equation}
P(spam|words) \propto P(words|spam)P(spam)
\end{equation}

\begin{equation}
P(ham|words) \propto P(words|ham)P(ham)
\end{equation}

For now, we assume $P(spam)=P(ham)=0.5$. Because we make this assumption without being certain that it is correct, we call this model Naive Bayes.

According to Conway and White :

"To calculate the conditional probability of a message, we combine the probabilities of
each term in the training data by taking their product. For example, if the frequency of
seeing html in a spam message is 0.30 and the frequency of seeing table in a spam
message is 0.10, then we’ll say that the probability of seeing both in a spam message is
0.30 × 0.10 = 0.03. But for those terms in the email that are not in our training data,
we have no information about their frequency in either spam or ham messages. 

One possible solution would be to assume that because we have not seen a term yet, its
probability of occurring in a certain class is zero. This, however, is very misguided.
First, it is foolish to assume that we will never see a term in the entire universe of spam
and ham simply because we have not yet seen it. Moreover, because we calculate conditional probabilities using products, if we assigned a zero probability to terms not in
our training data, elementary arithmetic tells us that we would calculate zero as the
probability of most messages, because we would be multiplying all the other probabilities by zero every time we encountered an unknown term. This would cause
catastrophic results for our classifier because many, or even all, messages would be
incorrectly assigned a zero probability of being either spam or ham.

Researchers have come up with many clever ways of trying to get around this problem,
such as drawing a random probability from some distribution or using natural language
processing (NLP) techniques to estimate the “spamminess” of a term given its context.
For our purposes, we will use a very simple rule: assign a very small probability to terms
that are not in the training set. This is, in fact, a common way of dealing with missing
terms in simple text classifiers"

Conway and White set this small probability to $0.0001\%$ for the use case in their book "Machine Learning for Hackers". However, this probability might be too large or too small in other cases. Here, we use $0.01\%$, because we discovered through trial and error that this probability yields better results.

In [9]:
classify.email <- function(msg, training.df, prior=0.5, c= 1e-4) {
 msg.tdm <- get.tdm(msg) # get the email message
 msg.freq <- rowSums(as.matrix(msg.tdm)) # calculate term frequency
 # Find intersections of words between TDM and message
 msg.match <- intersect(names(msg.freq), training.df$term)  
 if(length(msg.match) < 1) {
 return(prior*c^(length(msg.freq)))
 }
 else {
 match.probs <- training.df$occurrence[match(msg.match, training.df$term)]
 return(prior * prod(match.probs) * c^(length(msg.freq)-length(msg.match)))
 }
}

The `spam.classifier` function calculates the proportional probability of an email being spam or ham for each email and returns 1 if it is classified as spam and 0 if it is classified as ham.

In [10]:
spam.classifier <- function(docVec) {
 pr.spam <- classify.email(docVec, spam.df) # calculate spam probability
 pr.ham <- classify.email(docVec, ham.df) # calculate ham probability
 return(ifelse(pr.spam > pr.ham, 1, 0))
}
# returns 1 if email is classified as spam

We apply `spam.classifier` to test ham and test spam data.

In [11]:
spamClassification <- sapply(spam.test, spam.classifier)
hamClassification <- sapply(ham.test, spam.classifier)

Finally, we create a data frame with classification results.

In [12]:
classificationDf <- data.frame(
spam = c(length(spamClassification) - sum(spamClassification), sum(spamClassification)),
ham = c(length(hamClassification) - sum(hamClassification), sum(hamClassification))
)

row.names(classificationDf) <- c('Classified as ham', 'Classified as spam')

classificationDf

,spam,ham
,<dbl>,<dbl>
Classified as ham,43,692
Classified as spam,86,5


Here, we see the results in proportions.

In [13]:
prop.table(as.matrix(classificationDf),2)

,spam,ham
Classified as ham,0.3333333,0.992826399
Classified as spam,0.6666667,0.007173601


Our algorithm identified spam mail correctly in approximately $67\%$ of cases. Ham mail was classified correctly in almost all cases.

# Next Steps

There are a few possible steps in which we could improve this algoritm. I will add them in following versions of this article.

We can use bootstraping to find out the best value for `c`, the "small probability" mentioned above. 

We can also try using a non-naive Bayes algorithm, that is, an algoritm in which $P(spam) \neq P(ham)$.

We can also use external data to identify blacklisted sender emails or IP.